In [66]:
import re
import spacy
import textract
import os
from spacy.matcher import Matcher
import glob
import os
import random
import pandas as pd
import json

resumes_dict = {
    'resume_id':
    {                  #filename for now but to be replaced with unique id
        'name': ('first name', 'last name'),
        'email': ('email', 'position_tuple'),
        'phone': ('phone', 'position_tuple'),
        'dob': ('dob', 'position_tuple'),
        'address': ('address', 'position_tuple'),
        'skills': [('skill1', 'position_tuple1'), ('skill2', 'position_tuple2')],
    }                     
}

In [67]:
file_list = glob.glob(os.path.join(os.getcwd(), "/home/anudeepadi/Documents/Fw__Sample_resumes_", "*.docx"))

In [68]:
file_list

['/home/anudeepadi/Documents/Fw__Sample_resumes_/Resume1_UI.docx',
 '/home/anudeepadi/Documents/Fw__Sample_resumes_/Resume-Dileep2020 (1).docx',
 '/home/anudeepadi/Documents/Fw__Sample_resumes_/Resume_11_Dotnet_Developer_.docx',
 '/home/anudeepadi/Documents/Fw__Sample_resumes_/resume_8_service_now.docx',
 '/home/anudeepadi/Documents/Fw__Sample_resumes_/Resume4_Devops_engg.docx',
 '/home/anudeepadi/Documents/Fw__Sample_resumes_/Resume2_Java_full_stack.docx',
 '/home/anudeepadi/Documents/Fw__Sample_resumes_/Resume_9_cyber_security_.docx',
 '/home/anudeepadi/Documents/Fw__Sample_resumes_/Resume_12_backend_developer_.docx',
 '/home/anudeepadi/Documents/Fw__Sample_resumes_/Resume_6_data_engg.docx']

In [89]:
class parse:
    def __init__(self, file_list, nlp):
        self.file_list = file_list
        self.resumes = [textract.process(file_path).decode() for file_path in file_list]
        self.nlp = nlp

    def extract(self):
        ph_list, email_list, dob_list, names_list, location_list, skills_list, skillset_exp = [], [], [], [], [], [], []
        for resume in self.resumes:
            items = re.finditer(r'\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4}', resume)
            ph_span = [(resume[item.span()[0]:item.span()[1]], item.span()) for item in items]
            ph_list.append(ph_span)
            items = re.finditer(r'[\w\.-]+@[\w\.-]+', resume)
            em_span = [(resume[item.span()[0]:item.span()[1]], item.span()) for item in items]
            email_list.append(em_span)
            result2 = re.finditer(r"[\d]{1,2}-[\d]{1,2}-[\d]{4}", resume) # DD-MM-YYYY
            dob_span = [(resume[item.span()[0]:item.span()[1]], item.span()) for item in result2]
            dob_list.append(dob_span)
            doc = self.nlp(resume)
            list_ls, list_ns, list_ss = [], [], []
            for ent in doc.ents:
                if ent.label_ == 'PERSON':
                    es_names = (ent.text, ent.start_char, ent.end_char)
                    list_ns.append(es_names)
                if ent.label_ in ['GPE', 'LOC']:
                    es_locs = (ent.text, ent.start_char, ent.end_char)
                    list_ls.append(es_locs) 
                if ent.label_ in ['PRODUCT', 'ORG']:
                    s_span = (ent.text, ent.start_char, ent.end_char)
                    list_ss.append(s_span)
            location_list.append(list_ls)
            names_list.append(list_ns)
            skillset_exp.append(list_ss)
            noun_chunks = doc.noun_chunks
            # tokens = [token.text for token in nlp_text if not token.is_stop]
            tokens = [token.text for token in doc if token.is_stop]
            data = pd.read_csv("skills.csv") 
            skills = list(data.columns.values)

            skillset = []
            for token in tokens:
                if token.lower() in skills:
                    skillset.append(token)

            for token in noun_chunks:
                token = token.text.lower().strip()
                if token in skills:
                    skillset.append(token)

            st = list(set([i.capitalize() for i in set([i.lower() for i in skillset])]))
            skills_list.append(st)
        return ph_list, email_list, dob_list, names_list, location_list, skills_list, skillset_exp 
    
    def dictify(self):
        dict_ = {}
        ph_list, email_list, dob_list, names_list, location_list, skills_list, skillset_exp = self.extract()
        for idx, ph, email, dob, name, location, skills, skills_exp in zip(
            [file.split('.')[-2].split('/')[-1] for file in self.file_list], 
            ph_list, 
            email_list, 
            dob_list, 
            names_list, 
            location_list, 
            skills_list,
            skillset_exp
        ):
            dict_[idx] = {}
            if len(dob):
                dict_[idx]['dob'] = dob[0]
            else:
                dict_[idx]['dob'] = (None, None)
            if len(name):
                dict_[idx]['name'] = name[0]
            else:
                dict_[idx]['name'] = (None, None)
            if len(email): 
                dict_[idx]['email'] = email[0]
            else:
                dict_[idx]['email'] = (None, None)
            if len(ph):
                dict_[idx]['phone'] = ph[0]
            else:
                dict_[idx]['phone'] = (None, None)
            if len(location):
                dict_[idx]['location'] = location[0]
            else:
                dict_[idx]['location'] = (None, None)
            dict_[idx]['skills'] = skills
            dict_[idx]['skills_exp'] = skills_exp
        return dict_     

In [90]:
nlp = spacy.load('en_core_web_sm')
p = parse(file_list, nlp)

In [91]:
%%time
#ph_list, email_list, dob_list, names_list, location_list, skills_list, ents_ = p.extract()
resume_dict = p.dictify()

CPU times: user 5.96 s, sys: 112 ms, total: 6.07 s
Wall time: 6.07 s


In [92]:
with open('parsed_output2.json', 'w') as f:
    json.dump(resume_dict, f)